# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.getOrCreate()

In [3]:
df = spark.read.csv('customer_churn.csv', inferSchema=True, header=True)
df.head(3)

[Row(Names=u'Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location=u'10265 Elizabeth Mission Barkerburgh, AK 89518', Company=u'Harvey LLC', Churn=1),
 Row(Names=u'Kevin Mueller', Age=41.0, Total_Purchase=11916.22, Account_Manager=0, Years=6.5, Num_Sites=11.0, Onboard_date=datetime.datetime(2013, 8, 13, 0, 38, 46), Location=u'6157 Frank Gardens Suite 019 Carloshaven, RI 17756', Company=u'Wilson PLC', Churn=1),
 Row(Names=u'Eric Lozano', Age=38.0, Total_Purchase=12884.75, Account_Manager=0, Years=6.67, Num_Sites=12.0, Onboard_date=datetime.datetime(2016, 6, 29, 6, 20, 7), Location=u'1331 Keith Court Alyssahaven, DE 90114', Company=u'Miller, Johnson and Wallace', Churn=1)]

In [4]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



### VectorAssemble features

In [5]:
from pyspark.ml.feature import VectorAssembler

In [6]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [7]:
# choose numerical features except Account_Manager as it's assigned randomly
inputCols = ['Age',
             'Total_Purchase',
             'Years',
             'Num_Sites']

assembler = VectorAssembler(inputCols=inputCols, outputCol='features')
                                      
output = assembler.transform(df)

data = output.select('features', 'Churn')
data.show(3, False)

+-------------------------+-----+
|features                 |Churn|
+-------------------------+-----+
|[42.0,11066.8,7.22,8.0]  |1    |
|[41.0,11916.22,6.5,11.0] |1    |
|[38.0,12884.75,6.67,12.0]|1    |
+-------------------------+-----+
only showing top 3 rows



### Modeling

In [8]:
train, test = data.randomSplit([0.7, 0.3], seed=123)

In [9]:
from pyspark.ml.classification import LogisticRegression

In [10]:
lr = LogisticRegression(labelCol='Churn')
model = lr.fit(train)

In [11]:
model.summary.predictions.show(5)

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[22.0,11254.38,4....|    0|[4.80330322358869...|[0.99186412815737...|       0.0|
|[27.0,8628.8,5.3,...|    0|[5.64742992008906...|[0.99648582428088...|       0.0|
|[28.0,8670.98,3.9...|    0|[7.40287864517513...|[0.99939087554724...|       0.0|
|[28.0,9090.43,5.7...|    0|[1.90602416504468...|[0.87057182424028...|       0.0|
|[28.0,11128.95,5....|    0|[4.36055403871625...|[0.98738973965375...|       0.0|
+--------------------+-----+--------------------+--------------------+----------+
only showing top 5 rows



In [12]:
model.summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                633|                633|
|   mean|0.15639810426540285|0.11216429699842022|
| stddev| 0.3635195998705865|0.31581804295341376|
|    min|                  0|                0.0|
|    max|                  1|                1.0|
+-------+-------------------+-------------------+



### Evaluate the model

In [13]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [14]:
pred = model.evaluate(test)

In [15]:
pred.predictions.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|Churn|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[25.0,9672.03,5.4...|    0|[4.45728553683191...|[0.98853908393873...|       0.0|
|[26.0,8787.39,5.4...|    1|[1.07747513654642...|[0.74601587906872...|       0.0|
|[26.0,8939.61,4.5...|    0|[6.08134633024471...|[0.99772011168408...|       0.0|
|[28.0,11204.23,3....|    0|[1.69011971259467...|[0.84423990259824...|       0.0|
|[29.0,5900.78,5.5...|    0|[4.44595421204329...|[0.98840999172866...|       0.0|
|[29.0,11274.46,4....|    0|[4.64709674466340...|[0.99050168115979...|       0.0|
|[29.0,12711.15,5....|    0|[5.00285442680424...|[0.99332609877076...|       0.0|
|[29.0,13240.01,4....|    0|[6.70687780527334...|[0.99877901764305...|       0.0|
|[30.0,10183.98,5....|    0|[3.15571935401620...|[0.95913348972409...|       0.0|
|[30.0,11575.37,

In [16]:
# AUC
BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn').evaluate(pred.predictions)

0.7410130718954249

### Predict on new customers

In [17]:
new_df = spark.read.csv('new_customers.csv', inferSchema=True, header=True)
new_df.show()

+--------------+----+--------------+---------------+-----+---------+--------------------+--------------------+----------------+
|         Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|        Onboard_date|            Location|         Company|
+--------------+----+--------------+---------------+-----+---------+--------------------+--------------------+----------------+
| Andrew Mccall|37.0|       9935.53|              1| 7.71|      8.0|2011-08-29 18:37:...|38612 Johnny Stra...|        King Ltd|
|Michele Wright|23.0|       7526.94|              1| 9.28|     15.0|2013-07-22 18:19:...|21083 Nicole Junc...|   Cannon-Benson|
|  Jeremy Chang|65.0|         100.0|              1|  1.0|     15.0|2006-12-11 07:48:...|085 Austin Views ...|Barron-Robertson|
|Megan Ferguson|32.0|        6487.5|              0|  9.4|     14.0|2016-10-28 05:32:...|922 Wright Branch...|   Sexton-Golden|
|  Taylor Young|32.0|      13147.71|              1| 10.0|      8.0|2012-03-20 00:36:...|Unit 0789 Box 0

In [18]:
new_df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [19]:
new_data = assembler.transform(new_df)

In [20]:
model = lr.fit(data)

In [21]:
result = model.transform(new_data)

In [22]:
result.select('Names', 'Company', 'prediction').show()

+--------------+----------------+----------+
|         Names|         Company|prediction|
+--------------+----------------+----------+
| Andrew Mccall|        King Ltd|       0.0|
|Michele Wright|   Cannon-Benson|       1.0|
|  Jeremy Chang|Barron-Robertson|       1.0|
|Megan Ferguson|   Sexton-Golden|       1.0|
|  Taylor Young|        Wood LLC|       0.0|
| Jessica Drake|   Parks-Robbins|       1.0|
+--------------+----------------+----------+



We should assign account manager to those customers whose churn prediction is 1 (meaning stop buying service).